In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import newaxis
import scipy
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
from pyamg.classical import ruge_stuben_solver
from pyfem.sem import SEMhat
from pyfem.topo import Interval
norm = lambda x: np.max(np.abs(x)) if len(x)>0 else 0.0

# Setup mesh

In [ ]:
N  = 16
Ex = 4
Ey = 4

nx      = N+1
ny      = N+1
nx_dofs = N*Ex+1
ny_dofs = N*Ey+1
n_elem  = Ex*Ey

periodic = True

if periodic:
    nx_dofs -= 1
    ny_dofs -= 1
    
semh = SEMhat(N)

In [ ]:
def f(X):
    x = X[:,0]
    y = X[:,1]
    
    return np.sin(np.pi*x)*np.sin(np.pi*y)

def f2(X):
    x = X[:,0]
    y = X[:,1]
    
    return np.sin(np.pi*x)*np.sin(np.pi*y)*2*(np.pi)**2

def f(X):
    x = X[:,0]
    y = X[:,1]
    
    return np.cos(np.pi*x)*np.cos(np.pi*y)

def f2(X):
    x = X[:,0]
    y = X[:,1]
    
    return np.cos(np.pi*x)*np.cos(np.pi*y)*2*(np.pi)**2

sx = 0.1
def mx(chi, eta):
    
    return chi+sx*np.sin(np.pi*chi)*np.sin(np.pi*eta)

def dmx1(chi, eta):

    return np.ones_like(chi)+sx*np.pi*np.cos(np.pi*chi)*np.sin(np.pi*eta)

def dmx2(chi, eta):

    return np.zeros_like(chi)+sx*np.pi*np.sin(np.pi*chi)*np.cos(np.pi*eta)

sy = 0.1
def my(chi, eta):
    
    return eta+sy*np.sin(np.pi*chi)*np.sin(np.pi*eta)

def dmy1(chi, eta):

    return np.zeros_like(eta)+sy*np.pi*np.cos(np.pi*chi)*np.sin(np.pi*eta)

def dmy2(chi, eta):

    return np.ones_like(eta)+sy*np.pi*np.sin(np.pi*chi)*np.cos(np.pi*eta)

In [ ]:
topo  = Interval()

vertices = np.linspace(-1, 1, Ex+1)
etvx      = np.zeros((Ex, 2), dtype=np.int)
etvx[:,0] = np.arange(Ex)
etvx[:,1] = np.arange(Ex)+1
xq = topo.ref_to_phys(vertices[etvx], semh.xgll)
jacb_det0x = topo.calc_jacb(vertices[etvx])[0]
if periodic:
    xq = xq.ravel()[:-1]

vertices = np.linspace(-1, 1, Ey+1)
etvy      = np.zeros((Ey, 2), dtype=np.int)
etvy[:,0] = np.arange(Ey)
etvy[:,1] = np.arange(Ey)+1
yq = topo.ref_to_phys(vertices[etvy], semh.xgll)
jacb_det0y = topo.calc_jacb(vertices[etvy])[0]
if periodic:
    yq = yq.ravel()[:-1]

In [ ]:
# Build restriction operator
if periodic:
    R0x = sps.eye(nx_dofs)
    R0y = sps.eye(ny_dofs)
else:
    R0x = sps.dia_matrix((np.ones(nx_dofs),1),
                          shape=(nx_dofs-2,nx_dofs))

    R0y = sps.dia_matrix((np.ones(ny_dofs),1),
                         shape=(ny_dofs-2,ny_dofs))

R = sps.kron(R0y, R0x)

rngx = np.arange(nx_dofs)
rngy = np.arange(ny_dofs)

if not periodic:
    boundary_dofs = np.hstack([rngx,
                               rngx+nx_dofs*(ny_dofs-1),
                               rngy[1:-1]*nx_dofs])
    boundary_dofs = np.hstack([boundary_dofs,
                               rngy[1:-1]*nx_dofs+nx_dofs-1])
else:
    boundary_dofs = np.array([],dtype=np.int)

boundary_dofs.sort()

In [ ]:
dx = np.unique(xq)
dy = np.unique(yq)
X, Y = np.meshgrid(dx, dy)
dof_ref = np.zeros((len(X.ravel()), 2))
dof_ref[:,0] = X.ravel()
dof_ref[:,1] = Y.ravel()

dof_phys = np.zeros_like(dof_ref)
dof_phys[:,0] = mx(dof_ref[:,0], dof_ref[:,1])
dof_phys[:,1] = my(dof_ref[:,0], dof_ref[:,1])

In [ ]:
ax = np.repeat(semh.wgll[newaxis,:-1], Ex, axis=0)
ax = ax.ravel()
if not periodic:
    ax = np.hstack([ax,ax[0]])
ay = np.repeat(semh.wgll[newaxis,:-1], Ey, axis=0)
ay = ay.ravel()
if not periodic:
    ay = np.hstack([ay,ay[0]])

wvals  = ay[:,newaxis]*ax[newaxis,:]
wvals *= jacb_det0x*jacb_det0y
wvals  = wvals.ravel()

In [ ]:
# Make Q
etdx = np.arange(Ex*(N+1))
etdx = etdx.reshape((Ex, -1))
etdx -= np.arange(Ex)[:,newaxis]
if periodic:
    etdx[-1,-1] = etdx[0,0]
    
etdy = np.arange(Ey*(N+1))
etdy = etdy.reshape((Ey, -1))
etdy -= np.arange(Ey)[:,newaxis]
if periodic:
    etdy[-1,-1] = etdy[0,0]

cols = etdx.ravel()
rows = np.arange(len(cols))
vals = np.ones(len(cols))
Q0x = sps.coo_matrix((vals,(rows,cols))).tocsr()

cols = etdy.ravel()
rows = np.arange(len(cols))
vals = np.ones(len(cols))
Q0y = sps.coo_matrix((vals,(rows,cols))).tocsr()

Q = sps.kron(Q0y, Q0x)

a = Q.dot(np.arange(nx_dofs*ny_dofs)).reshape((ny*Ey,nx*Ex))
etd = np.zeros((Ex*Ey, nx*ny), dtype=np.int)
ind = 0
for iy in range(Ey):
    for ix in range(Ex):
        indy = iy*ny
        indx = ix*nx
        etd[ind,:] = a[indy:indy+ny,indx:indx+nx].ravel()
        ind += 1
        
cols = etd.ravel()
rows = np.arange(len(cols))
vals = np.ones(len(cols))
Q = sps.coo_matrix((vals,(rows,cols))).tocsr()

In [ ]:
# Compute jacobian information
dr = dof_ref

Jacb = np.zeros((dr.shape[0],2,2,))
Jacb[:,0,0] = dmx1(dr[:,0], dr[:,1])
Jacb[:,0,1] = dmx2(dr[:,0], dr[:,1])
Jacb[:,1,0] = dmy1(dr[:,0], dr[:,1])
Jacb[:,1,1] = dmy2(dr[:,0], dr[:,1])

jacb_det = np.linalg.det(Jacb)
jacb_det = jacb_det.ravel()

Jacb_inv = np.zeros_like(Jacb)
Jacb_inv[:,0,0] =  Jacb[:,1,1]
Jacb_inv[:,1,1] =  Jacb[:,0,0]
Jacb_inv[:,0,1] = -Jacb[:,0,1]
Jacb_inv[:,1,0] = -Jacb[:,1,0]
Jacb_inv = Jacb_inv/jacb_det.reshape((Jacb.shape[0],1,1))

# for a, b in zip(Jacb, Jacb_inv):
#     print norm(a.dot(b)-np.eye(2)), norm(b.dot(a)-np.eye(2))

# Poisson

In [ ]:
# build Gij
G0 = np.zeros_like(Jacb)
assert len(Jacb)==len(wvals)

for i in range(len(Jacb)):
    G0[i,:,:]  = Jacb_inv[i].dot(Jacb_inv[i].T)
    G0[i,:,:] *= wvals[i]*jacb_det[i]   

G11 = []
G12 = []
G21 = []
G22 = []

nn = nx*ny
s  = (nn, nn)
for i in range(n_elem):
    G11 += [sps.dia_matrix((G0[etd[i],0,0], 0), shape=s)]
    G12 += [sps.dia_matrix((G0[etd[i],0,1], 0), shape=s)]
    G21 += [sps.dia_matrix((G0[etd[i],1,0], 0), shape=s)]
    G22 += [sps.dia_matrix((G0[etd[i],1,1], 0), shape=s)]

In [ ]:
# Build poisson stiffness matrix A

D1 = sps.kron(sps.eye(ny), semh.Dh)/jacb_det0x
D2 = sps.kron(semh.Dh, sps.eye(nx))/jacb_det0y

A0a = []
for i in range(n_elem):
    A0a += [D1.T.dot(G11[i].dot(D1)+G12[i].dot(D2))+\
            D2.T.dot(G21[i].dot(D1)+G22[i].dot(D2))]
A0 = sps.block_diag(A0a).tocsr()
A0 = Q.T.dot(A0.dot(Q))
A  = R.dot(A0.dot(R.T))

# Build mass matrix B
nd = nx_dofs*ny_dofs
b = Q.T.dot((wvals*jacb_det)[etd.ravel()])
#Bl = sps.dia_matrix((wvals*jacb_det, 0), shape=(nd,nd))
# Bl is not the local mass matrix.
# I am just using bad notation here
Bl = sps.dia_matrix((b, 0), shape=(nd,nd))
Binv_data = (1.0/Bl.data).ravel()
Binv_data = R.dot(Binv_data)

if nd<=1e3:
    print np.min(np.linalg.svd(A.toarray())[1])

In [ ]:
fh  = f2(dof_phys)
fl = fh
rhs = Bl.dot(fl)
radj = np.zeros(nx_dofs*ny_dofs)
radj[boundary_dofs] = f(dof_phys)[boundary_dofs]
rhs = R.dot(rhs-A0.dot(radj))

if periodic:
    rhs -= np.mean(rhs)
    
ml = ruge_stuben_solver(A)
residuals = []
sol = R.T.dot(ml.solve(rhs, tol=1e-14, 
                       maxiter=500, residuals=residuals,
                       accel='cg'))
sol[boundary_dofs] = f(dof_phys)[boundary_dofs]

if periodic:
    sol -= np.mean(sol)

print len(residuals), residuals[-1]
print 
print norm(f(dof_phys)-sol)/norm(f(dof_phys))

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
s = (ny_dofs,nx_dofs)
X, Y = dof_phys[:,0], dof_phys[:,1]
X = X.reshape(s)
Y = Y.reshape(s)
ax.plot_wireframe(X, Y, f(dof_phys).reshape(s))
ax.plot_wireframe(X, Y, sol.reshape(s),
                  color='g')
plt.savefig("sol.pdf")

In [ ]:
plt.scatter(dof_phys[:,0], dof_phys[:,1])